# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [26]:
# load data from database
engine = create_engine('sqlite:///Messages.db')

In [27]:
engine.table_names()

['Messages']

In [28]:
df = pd.read_sql('SELECT * FROM "Messages"', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26215 entries, 0 to 26214
Data columns (total 39 columns):
message                   26215 non-null object
original                  10170 non-null object
genre                     26215 non-null object
related                   26215 non-null int64
request                   26215 non-null int64
offer                     26215 non-null int64
aid_related               26215 non-null int64
medical_help              26215 non-null int64
medical_products          26215 non-null int64
search_and_rescue         26215 non-null int64
security                  26215 non-null int64
military                  26215 non-null int64
child_alone               26215 non-null int64
water                     26215 non-null int64
food                      26215 non-null int64
shelter                   26215 non-null int64
clothing                  26215 non-null int64
money                     26215 non-null int64
missing_people            26215 non-null i

In [29]:
X = df[['message','genre']]
Y = df.drop(columns=['message','original','genre'])

In [30]:
X = X.merge(pd.get_dummies(X['genre']),left_index=True, right_index=True).drop(columns=['genre'])

In [32]:
X.head()

,message,direct,news,social
0,Weather update - a cold front from Cuba that c...,1,0,0
1,Is the Hurricane over or is it not over,1,0,0
2,Looking for someone but no name,1,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,1,0,0
4,"says: west side of Haiti, rest of the country ...",1,0,0


In [7]:
X['message'] = X['message'].str.lower()

In [8]:
X.head()

,message,direct,news,social
0,weather update - a cold front from cuba that c...,1,0,0
1,is the hurricane over or is it not over,1,0,0
2,looking for someone but no name,1,0,0
3,un reports leogane 80-90 destroyed. only hospi...,1,0,0
4,"says: west side of haiti, rest of the country ...",1,0,0


In [9]:
X['message'].apply(lambda x: re.sub('[^\w\s]','',x))

0        weather update  a cold front from cuba that co...
1                  is the hurricane over or is it not over
2                          looking for someone but no name
3        un reports leogane 8090 destroyed only hospita...
4        says west side of haiti rest of the country to...
5                    information about the national palace
6                           storm at sacred heart of jesus
7        please we need tents and water we are in silo ...
8           i would like to receive the messages thank you
9        i am in croixdesbouquets we have health issues...
10       theres nothing to eat and water we starving an...
11       i am in petionville i need more information re...
12       i am in thomassin number 32 in the area named ...
13       lets do it together need food in delma 75 in d...
14       more information on the 4636 number in order f...
15       a comitee in delmas 19 rue  street  janvier im...
16       we need food and water in klecin 12 we are dyi.

### 2. Write a tokenization function to process your text data

In [33]:
def tokenize(text):
    tokens = word_tokenize(re.sub('[^\w\s]',' ',text))
    lem = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lem.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
       
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:

pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X['message'],Y,test_size=0.3,random_state = 208)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [36]:
X_train.head()

7727     21 donations Ephesians 4:10,11 Roman12:6 1cor1...
10446    Thousands Feared Dead as Major Quake Strikes H...
6033     .. still in Cit Militaire 1 for six people. Pl...
6671     (..) victim(s). If the earthquake happens in S...
16463    An active surveillance for communicable diseas...
Name: message, dtype: object

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_predict = pipeline.predict(X_test)

In [14]:
Y_test.values[:,0].shape

(7865,)

In [15]:
for i in range(Y_test.shape[1]):
    print(Y_test.columns[i])
    print(classification_report(Y_predict[:,i],Y_test.values[:,i]))

related
             precision    recall  f1-score   support

          0       0.37      0.59      0.46      1150
          1       0.92      0.82      0.87      6663
          2       0.38      0.46      0.42        52

avg / total       0.84      0.79      0.80      7865

request
             precision    recall  f1-score   support

          0       0.99      0.88      0.93      7301
          1       0.35      0.85      0.50       564

avg / total       0.94      0.88      0.90      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7865
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      7865

aid_related
             precision    recall  f1-score   support

          0       0.87      0.72      0.79      5584
          1       0.52      0.74      0.61      2281

avg / total       0.77      0.73      0.74      7865

medical_help
             precision    recall  f1-sco

C:\Users\thoma\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'text_pipeline', 'clf', 'text_pipeline__memory', 'text_pipeline__steps', 'text_pipeline__vect', 'text_pipeline__tfidf', 'text_pipeline__vect__analyzer', 'text_pipeline__vect__binary', 'text_pipeline__vect__decode_error', 'text_pipeline__vect__dtype', 'text_pipeline__vect__encoding', 'text_pipeline__vect__input', 'text_pipeline__vect__lowercase', 'text_pipeline__vect__max_df', 'text_pipeline__vect__max_features', 'text_pipeline__vect__min_df', 'text_pipeline__vect__ngram_range', 'text_pipeline__vect__preprocessor', 'text_pipeline__vect__stop_words', 'text_pipeline__vect__strip_accents', 'text_pipeline__vect__token_pattern', 'text_pipeline__vect__tokenizer', 'text_pipeline__vect__vocabulary', 'text_pipeline__tfidf__norm', 'text_pipeline__tfidf__smooth_idf', 'text_pipeline__tfidf__sublinear_tf', 'text_pipeline__tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimato

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters4 = {'clf__estimator__max_depth': [500, 750, None],
 'clf__estimator__max_features': ['auto'],
 'clf__estimator__min_samples_leaf': [1, 3],
 'clf__estimator__min_samples_split': [2, 5, 10]}

In [19]:
cv4 = GridSearchCV(pipeline,parameters4)
cv4.fit(X_train,Y_train)
cv4.best_params_

{'clf__estimator__max_depth': 500,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2}

In [25]:
parameters5 = {'clf__estimator__max_depth': [500,600, None],
 'clf__estimator__max_features': ['auto'],
 'clf__estimator__min_samples_leaf': [1, 3],
 'clf__estimator__min_samples_split': [2,3],
 'clf__estimator__n_estimators': [200, 1000, 2000]}

In [26]:
cv5 = GridSearchCV(pipeline,parameters5)
cv5.fit(X_train,Y_train)
cv5.best_params_

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
nt

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [91]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(re.sub('[^\w\s]',' ',text))
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try: 
                first_word, first_tag = pos_tags[0]
            except:
                print(sentence_list,sentence,pos_tags)
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

## Try adjectives possibly phrases
class StartingAdjExtractor(BaseEstimator, TransformerMixin):

    def starting_adj(self, text):
        sentence_list = nltk.sent_tokenize(re.sub('[^\w\s]',' ',text))
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['JJ'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_adj)
        return pd.DataFrame(X_tagged)

## Try NN classifier later?

In [92]:
pipeline2 = Pipeline([
    ('features',FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('starting_verb', StartingVerbExtractor()),
        ('starting_adj', StartingAdjExtractor())
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [39]:
def starting_verb(self, text):
    sentence_list = nltk.sent_tokenize(re.sub('[^\w\s]',' ',text))
    for sentence in sentence_list:
        pos_tags = nltk.pos_tag(tokenize(sentence))
        try: 
            first_word, first_tag = pos_tags[0]
        except:
            print(sentence_list,sentence,pos_tags)
        if first_tag in ['VB', 'VBP'] or first_word == 'RT':
            return True
    return False

In [54]:
pd.Series(pd.DataFrame(X_train))

ValueError: Wrong number of items passed 1, placement implies 18350

In [93]:
parameters4 = {'clf__estimator__max_depth': [500],
 'clf__estimator__max_features': ['auto'],
 'clf__estimator__min_samples_leaf': [1],
 'clf__estimator__min_samples_split': [2]}

In [94]:
cv4 = GridSearchCV(pipeline2,parameters4)
cv4.fit(X_train,Y_train)
cv4.best_params_

{'clf__estimator__max_depth': 500,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 2}

In [66]:
blahiter(X_train)

In [71]:
text = next(blah1)

In [72]:
text

'.. still in cit militaire 1 for six people. please help me, hurricanes .. '

In [73]:
sent = nltk.sent_tokenize(text)
print(sent)

['.. still in cit militaire 1 for six people.', 'please help me, hurricanes ..']


In [80]:
tokenize(sent[1])

['please', 'help', 'me', 'hurricane']

In [75]:
tags = nltk.pos_tag(tokenize(sent[1]))
print(tags)

[('please', 'VB'), ('help', 'VB'), ('me', 'PRP'), ('hurricane', 'VB')]


In [46]:
f,t = tags[0]

In [48]:
t

'DT'

In [76]:
SVE = StartingVerbExtractor()

In [77]:
SVE.starting_verb(text)

True

### 9. Export your model as a pickle file

In [18]:
import pickle

In [96]:
import pickle
def save_model(model, model_filepath):
    """Save model as pickle file"""
    pickle.dump(model, open(model_filepath, 'wb'))
    
def load_model(model_filepath):
    """load model as pickle file"""
    return pickle.load(open(model_filepath, 'rb'))



In [97]:
save_model(cv4, 'model4_2.pickle')

In [98]:
cv4load  = load_model('model4.pickle')

In [99]:
cv4_2load  = load_model('model4_2.pickle')

In [100]:
Y_predict = cv4_2load.predict(X_test)

In [101]:
for i in range(Y_test.shape[1]):
    print(Y_test.columns[i])
    print(classification_report(Y_predict[:,i],Y_test.values[:,i]))

related
             precision    recall  f1-score   support

          0       0.37      0.63      0.47      1076
          1       0.93      0.82      0.87      6737
          2       0.40      0.48      0.43        52

avg / total       0.85      0.80      0.82      7865

request
             precision    recall  f1-score   support

          0       0.98      0.88      0.93      7266
          1       0.36      0.83      0.51       599

avg / total       0.94      0.88      0.90      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7865
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      7865

aid_related
             precision    recall  f1-score   support

          0       0.88      0.73      0.80      5589
          1       0.53      0.76      0.63      2276

avg / total       0.78      0.74      0.75      7865

medical_help
             precision    recall  f1-sco

C:\Users\thoma\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


             precision    recall  f1-score   support

          0       1.00      0.87      0.93      7815
          1       0.02      0.50      0.05        50

avg / total       0.99      0.87      0.92      7865

infrastructure_related
             precision    recall  f1-score   support

          0       1.00      0.94      0.97      7856
          1       0.01      0.44      0.02         9

avg / total       1.00      0.94      0.97      7865

transport
             precision    recall  f1-score   support

          0       1.00      0.96      0.98      7828
          1       0.08      0.76      0.14        37

avg / total       0.99      0.96      0.97      7865

buildings
             precision    recall  f1-score   support

          0       1.00      0.95      0.97      7827
          1       0.07      0.76      0.13        38

avg / total       0.99      0.95      0.97      7865

electricity
             precision    recall  f1-score   support

          0       1.00      0.9

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
def load_data(database_filepath):
    dbpath = 'sqlite:///' + database_filepath
    engine = create_engine(dbpath)
    df = pd.read_sql('SELECT * FROM "Messages"', engine)
    X = df[['message','genre']]
    Y = df.drop(columns=['message','original','genre'])

    return X,Y, list(Y.columns)


In [3]:
def tokenize(text):
    tokens = word_tokenize(re.sub('[^\w\s]',' ',text))
    lem = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lem.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
       
    return clean_tokens

In [15]:
def build_model():
    
    class StartingVerbExtractor(BaseEstimator, TransformerMixin):

        def starting_verb(self, text):
            sentence_list = nltk.sent_tokenize(re.sub('[^\w\s]',' ',text))
            for sentence in sentence_list:
                pos_tags = nltk.pos_tag(tokenize(sentence))
                try: 
                    first_word, first_tag = pos_tags[0]
                except:
                    print(sentence_list,sentence,pos_tags)
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            return False

        def fit(self, x, y=None):
            return self

        def transform(self, X):
            X_tagged = pd.Series(X).apply(self.starting_verb)
            return pd.DataFrame(X_tagged)
        pass

    class StartingAdjExtractor(BaseEstimator, TransformerMixin):
        def starting_adj(self, text):
            sentence_list = nltk.sent_tokenize(re.sub('[^\w\s]',' ',text))
            for sentence in sentence_list:
                pos_tags = nltk.pos_tag(tokenize(sentence))
                first_word, first_tag = pos_tags[0]
                if first_tag in ['JJ'] or first_word == 'RT':
                    return True
            return False

        def fit(self, x, y=None):
            return self

        def transform(self, X):
            X_tagged = pd.Series(X).apply(self.starting_adj)
            return pd.DataFrame(X_tagged)
        pass
    
    pipeline = Pipeline([
        ('features',FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor()),
        ])),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])
    
    parameters = {'clf__estimator__max_depth': [500, 750, None],
    'clf__estimator__max_features': ['auto'],
    'clf__estimator__min_samples_leaf': [1, 3],
    'clf__estimator__min_samples_split': [2, 5, 10]}

    cv = GridSearchCV(pipeline,parameters)
    
    return cv


In [16]:
def evaluate_model(model, X_test, Y_test, category_names):
    Y_predict = model.predict(X_test)
    for i in range(Y_test.shape[1]):
        print(category_names[i])
        print(classification_report(Y_predict[:,i],Y_test.values[:,i]))
    pass

In [17]:
def save_model(model, model_filepath):
    pickle.dump(model, open(model_filepath, 'wb'))
    pass

In [18]:
X, Y, category_names = load_data("Messages.db")

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [56]:
model = build_model()

In [57]:
model.fit(X_test,Y)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [500, 750, None], 'clf__estimator__max_features': ['auto'], 'clf__estimator__min_samples_leaf': [1, 3], 'clf__estimator__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)